In [1]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from io import StringIO
import pygwalker as pyg
import os
from tqdm import tqdm
import polars.selectors as cs
from scipy.stats import zscore 

reading competition editions json

In [5]:
ce_json_path = './skillcorner-competition-editions.json'
ce_df1 = pl.read_json(ce_json_path)
ce_df1.head(1)
#ce_df.columns

count,next,previous,results
i64,null,null,list[struct[4]]
938,null,null,"[{0,{0,""FIFA"",""World Cup"",""male"",""adult""},{10,2014,2014,""2014""},""FIFA - World Cup - 2014""}, {1,{1,""ENG"",""Premier League"",""male"",""adult""},{2,2015,2016,""2015/2016""},""ENG - Premier League - 2015/2016""}, … {1129,{221,""INT"",""Pinatar National Team Friendlies"",""male"",""U21""},{128,2025,2025,""2025""},""INT - Pinatar National Team Friendlies - 2025""}]"


In [6]:
#ce_df2 = ce_df1.explode('results')
ce_df2 = ce_df1.select('results').explode('results').unnest('results')
#ce_df2 = ce_df2.unnest('results')
ce_df2.head(1)

id,competition,season,name
i64,struct[5],struct[4],str
0,"{0,""FIFA"",""World Cup"",""male"",""adult""}","{10,2014,2014,""2014""}","""FIFA - World Cup - 2014"""


In [4]:
ce_name_id = ce_df2.select([
    pl.col('name'),
    pl.col('id'),
    ]
)
ce_name_id.head(1)

name,id
str,i64
"""FIFA - World Cup - 2014""",0


In [5]:
ce_competition = ce_df2.select('competition').unnest('competition')
ce_competition = ce_competition.rename({col: f"competition_{col}" for col in ce_competition.columns})
ce_competition.head(1)

competition_id,competition_area,competition_name,competition_gender,competition_age_group
i64,str,str,str,str
0,"""FIFA""","""World Cup""","""male""","""adult"""


In [6]:
ce_season = ce_df2.select('season').unnest('season')
ce_season = ce_season.rename({col: f"season_{col}" for col in ce_season.columns})
ce_season.head(1)

season_id,season_start_year,season_end_year,season_name
i64,i64,i64,str
10,2014,2014,"""2014"""


In [7]:
ce = pl.concat([
    ce_name_id,
    ce_season,
    ce_competition
],
how = "horizontal",)
ce.head(10)

name,id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""FRA - Ligue 1 - 2016/2017""",5,3,2016,2017,"""2016/2017""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""FRA - Ligue 1 - 2017/2018""",6,4,2017,2018,"""2017/2018""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""ESP - LaLiga - 2016/2017""",7,3,2016,2017,"""2016/2017""",4,"""ESP""","""LaLiga""","""male""","""adult"""
"""ESP - LaLiga - 2017/2018""",8,4,2017,2018,"""2017/2018""",4,"""ESP""","""LaLiga""","""male""","""adult"""


reading json player file- test on 1 file


In [8]:
position_json_testpath = './skillcorner-20250214/metrics-ce249-rAM.json'

position_test_df = pl.read_json(position_json_testpath)

In [9]:
position_test_df.columns
#position_test_df.head(10)

['Sprinting Distance TIP',
 'Count HSR TIP 1',
 'Count Medium Deceleration 1',
 'HSR Distance 1',
 'Sprinting Distance TIP 2',
 'Count Sprint 1',
 'Count High Deceleration TIP',
 'Minutes 2',
 'Count High Deceleration OTIP',
 'Minutes OTIP 2',
 'Count Sprint OTIP 2',
 'Running Distance 1',
 'Count High Deceleration 1',
 'competition_edition_id',
 'Count High Deceleration 2',
 'Count High Acceleration TIP 2',
 'Minutes',
 'HSR Distance',
 'Count High Acceleration',
 'Distance',
 'team_id',
 'Sprinting Distance TIP 1',
 'Count Medium Deceleration TIP 2',
 'HSR Distance OTIP',
 'Minutes TIP',
 'Running Distance OTIP 1',
 'short_name',
 'season_name',
 'Distance 2',
 'Count Medium Acceleration 2',
 'Count Medium Deceleration TIP 1',
 'player_id',
 'Sprinting Distance',
 'Count HSR OTIP',
 'Count High Acceleration OTIP 2',
 'date',
 'Count High Deceleration TIP 2',
 'Minutes OTIP 1',
 'Count HSR TIP 2',
 'HSR Distance OTIP 2',
 'HSR Distance OTIP 1',
 'Count HSR',
 'Count Medium Deceleratio

In [10]:
#define the path to the entire directory
json_dir_path = './skillcorner-20250214/'

### Look at correlations between parameters
Looking at the correlations across all competitions across all positions to see if we can leave out certain parameters

In [11]:
json_list = os.listdir(json_dir_path)
#json_list

In [12]:
"""dfs = []
for jsonf in json_list:
    json_path = f"{json_dir_path}{jsonf}"
    dfs.append(pl.read_json(json_path))

corr_df = pl.concat(dfs, how="align_full")"""

'dfs = []\nfor jsonf in json_list:\n    json_path = f"{json_dir_path}{jsonf}"\n    dfs.append(pl.read_json(json_path))\n\ncorr_df = pl.concat(dfs, how="align_full")'

In [13]:
#loop through the json files and concat them into one big df

# creating an empty data frame to concat the json dataframes to
read_json_list = []

for jsonf in tqdm(json_list):
    json_path = f"{json_dir_path}{jsonf}"
    json_df = pl.read_json(json_path)

    read_json_list.append(json_df)

corr_df = pl.concat(read_json_list)


corr_df.shape
#corr_df.head(2)
"""
output_path = "json_output.csv"
corr_df.write_csv(output_path)"""

100%|██████████| 1652/1652 [00:26<00:00, 62.84it/s] 


'\noutput_path = "json_output.csv"\ncorr_df.write_csv(output_path)'

In [14]:
pl.Config.set_tbl_cols(250)
corr_df.head()

Sprinting Distance TIP,Count HSR TIP 1,Count Medium Deceleration 1,HSR Distance 1,Sprinting Distance TIP 2,Count Sprint 1,Count High Deceleration TIP,Minutes 2,Count High Deceleration OTIP,Minutes OTIP 2,Count Sprint OTIP 2,Running Distance 1,Count High Deceleration 1,competition_edition_id,Count High Deceleration 2,Count High Acceleration TIP 2,Minutes,HSR Distance,Count High Acceleration,Distance,team_id,Sprinting Distance TIP 1,Count Medium Deceleration TIP 2,HSR Distance OTIP,Minutes TIP,Running Distance OTIP 1,short_name,season_name,Distance 2,Count Medium Acceleration 2,Count Medium Deceleration TIP 1,player_id,Sprinting Distance,Count HSR OTIP,Count High Acceleration OTIP 2,date,Count High Deceleration TIP 2,Minutes OTIP 1,Count HSR TIP 2,HSR Distance OTIP 2,HSR Distance OTIP 1,Count HSR,Count Medium Deceleration OTIP 2,Running Distance,Count Medium Acceleration TIP 2,match_name,team,Count Sprint TIP 1,Distance TIP 2,Count High Deceleration OTIP 1,Distance TIP 1,HSR Distance TIP 1,Running Distance TIP 1,quality_check,Count Medium Acceleration,Minutes TIP 2,Count High Deceleration TIP 1,Count High Acceleration 2,Count Medium Acceleration TIP,Count HSR 1,Count HSR OTIP 1,Count Medium Deceleration,Minutes OTIP,Minutes 1,Running Distance TIP,Count Medium Acceleration OTIP,Count Medium Deceleration OTIP,Count High Acceleration OTIP 1,PSV-99,Running Distance OTIP,Count Medium Deceleration TIP,Count Sprint OTIP 1,Count High Acceleration TIP 1,Distance OTIP 2,Sprinting Distance 2,Count High Acceleration OTIP,competition_id,team_name,Count High Deceleration,Count Medium Deceleration OTIP 1,Count Sprint OTIP,Count HSR 2,Running Distance TIP 2,HSR Distance TIP,HSR Distance 2,Count Sprint 2,Distance OTIP 1,Running Distance OTIP 2,Count Medium Acceleration TIP 1,Sprinting Distance 1,Count HSR OTIP 2,season_id,Count Sprint TIP,position,Count High Acceleration TIP,player_name,HSR Distance TIP 2,Count Medium Acceleration 1,Distance OTIP,group,player_birthdate,Distance 1,Count High Acceleration 1,Count HSR TIP,Count Sprint,Count Medium Deceleration 2,Sprinting Distance OTIP 2,Sprinting Distance OTIP 1,Count High Deceleration OTIP 2,Count Medium Acceleration OTIP 1,Minutes TIP 1,match_id,Running Distance 2,Count Sprint TIP 2,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Distance TIP
i64,i64,i64,i64,i64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,str,i64,f64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,bool,i64,f64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64
30,13,42,387,19,3,3,49.48,4,17.3,1,1354,8,249,3,0,96.63,745,4,13026,462,11,9,443,24.21,842,"""P. Gerkens""","""2021/2022""",6216,48,8,8701,148,34,0,"""2021-07-25""",1,18.27,6,218,225,59,17,2415,10,"""KV Mechelen v Royal Antwerp FC""","""Royal Antwerp FC""",1,1558,3,1874,139,369,true,100,11.08,2,1,22,33,17,89,35.57,47.15,661,47,34,0,25.7,1504,17,1,0,2744,56,0,16,"""Royal Antwerp FC""",11,17,2,26,292,260,358,2,2983,662,12,92,17,8,2,"""AM""",0,"""Pieter Gerkens""",121,52,5727,"""Midfield""","""1995-02-17""",6810,3,19,5,47,18,27,1,24,13.13,44999,1061,1,23,45,3432
136,9,31,201,119,3,2,50.07,1,15.07,0,489,5,249,3,1,96.19,476,5,9969,27,17,11,167,25.52,178,"""X. Mercier""","""2021/2022""",5127,41,9,10932,202,17,1,"""2021-07-24""",1,14.52,11,63,104,39,14,1101,14,"""Oud-Heverlee Leuven v SV Zulte…","""Oud-Heverlee Leuven""",1,1775,1,1660,81,267,true,77,12.32,1,2,25,20,10,77,29.59,46.12,610,32,25,0,28.3,402,20,1,1,1739,154,1,16,"""Oud-Heverlee Leuven""",8,11,1,19,343,265,275,5,1795,224,11,48,7,8,4,"""AM""",2,"""Xavier Mercier""",184,36,3534,"""Midfield""","""1989-07-25""",4842,3,20,8,46,0,24,0,15,13.2,45002,612,3,17,24,3435
26,13,57,292,0,4,2,24.95,3,6.57,1,714,7,249,4,1,71.07,421,3,8709,8,26,11,167

In [15]:
nonull_numeric = corr_df.drop_nulls().select(cs.numeric())

In [16]:
correlations = nonull_numeric.corr()

In [17]:
# only retaining meaningfull correlation

# Initialize an empty list to store the filtered correlation pairs
filtered_data = []

# Iterate over the rows and columns to extract variable pairs and correlations
for i, row in enumerate(correlations.columns):
    for j, col in enumerate(correlations.columns):
        if i < j:  # Skip self-correlation and duplicate pairs (ensure we only process one pair)
            corr_value = correlations[row][j]  # Access the correlation value at row i, column j
            if corr_value > 0.8:  # Only keep correlations > 0.8
                filtered_data.append([row, col, corr_value])

# Create a Polars DataFrame with the filtered data
filtered_df = pl.DataFrame(filtered_data, schema=["Variable 1", "Variable 2", "Correlation"])

# Write the filtered data to a CSV file (since Polars doesn't support writing to Excel directly)
filtered_df.write_excel(workbook = "filtered_correlations.xlsx")

print("Filtered correlations saved to 'filtered_correlations.csv'.")

Filtered correlations saved to 'filtered_correlations.csv'.


/tmp/ipykernel_3858/36976019.py:15: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  filtered_df = pl.DataFrame(filtered_data, schema=["Variable 1", "Variable 2", "Correlation"])


In [18]:
nonull_numeric_np = nonull_numeric.to_numpy()

In [19]:
#get back the correlations
#get back the z-scores
z_scores = zscore(nonull_numeric_np)

In [20]:
# Find outliers
outliers = nonull_numeric_np[(np.abs(zscore(nonull_numeric_np)) >= 5).all(axis=1)]

# Check the shape
print(outliers.shape)

(0, 106)


In [21]:
#pyg.walk(position_test_df)

In [22]:
ce.head()

name,id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""


In [23]:
# change column name of ce df id collumn for easy join
ce = ce.rename({"id":"competition_edition_id"})

In [24]:
ce.head()

name,competition_edition_id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""


In [25]:
#join corr_df (competition_edition_id) and ce_df (id)
work_df = corr_df.join(ce, on = "competition_edition_id")

In [26]:
pl.Config.set_tbl_cols(250)
work_df.head()

Sprinting Distance TIP,Count HSR TIP 1,Count Medium Deceleration 1,HSR Distance 1,Sprinting Distance TIP 2,Count Sprint 1,Count High Deceleration TIP,Minutes 2,Count High Deceleration OTIP,Minutes OTIP 2,Count Sprint OTIP 2,Running Distance 1,Count High Deceleration 1,competition_edition_id,Count High Deceleration 2,Count High Acceleration TIP 2,Minutes,HSR Distance,Count High Acceleration,Distance,team_id,Sprinting Distance TIP 1,Count Medium Deceleration TIP 2,HSR Distance OTIP,Minutes TIP,Running Distance OTIP 1,short_name,season_name,Distance 2,Count Medium Acceleration 2,Count Medium Deceleration TIP 1,player_id,Sprinting Distance,Count HSR OTIP,Count High Acceleration OTIP 2,date,Count High Deceleration TIP 2,Minutes OTIP 1,Count HSR TIP 2,HSR Distance OTIP 2,HSR Distance OTIP 1,Count HSR,Count Medium Deceleration OTIP 2,Running Distance,Count Medium Acceleration TIP 2,match_name,team,Count Sprint TIP 1,Distance TIP 2,Count High Deceleration OTIP 1,Distance TIP 1,HSR Distance TIP 1,Running Distance TIP 1,quality_check,Count Medium Acceleration,Minutes TIP 2,Count High Deceleration TIP 1,Count High Acceleration 2,Count Medium Acceleration TIP,Count HSR 1,Count HSR OTIP 1,Count Medium Deceleration,Minutes OTIP,Minutes 1,Running Distance TIP,Count Medium Acceleration OTIP,Count Medium Deceleration OTIP,Count High Acceleration OTIP 1,PSV-99,Running Distance OTIP,Count Medium Deceleration TIP,Count Sprint OTIP 1,Count High Acceleration TIP 1,Distance OTIP 2,Sprinting Distance 2,Count High Acceleration OTIP,competition_id,team_name,Count High Deceleration,Count Medium Deceleration OTIP 1,Count Sprint OTIP,Count HSR 2,Running Distance TIP 2,HSR Distance TIP,HSR Distance 2,Count Sprint 2,Distance OTIP 1,Running Distance OTIP 2,Count Medium Acceleration TIP 1,Sprinting Distance 1,Count HSR OTIP 2,season_id,Count Sprint TIP,position,Count High Acceleration TIP,player_name,HSR Distance TIP 2,Count Medium Acceleration 1,Distance OTIP,group,player_birthdate,Distance 1,Count High Acceleration 1,Count HSR TIP,Count Sprint,Count Medium Deceleration 2,Sprinting Distance OTIP 2,Sprinting Distance OTIP 1,Count High Deceleration OTIP 2,Count Medium Acceleration OTIP 1,Minutes TIP 1,match_id,Running Distance 2,Count Sprint TIP 2,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Distance TIP,name,season_id_right,season_start_year,season_end_year,season_name_right,competition_id_right,competition_area,competition_name,competition_gender,competition_age_group
i64,i64,i64,i64,i64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,str,i64,f64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,bool,i64,f64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,str,i64,str,str,str,str
30,13,42,387,19,3,3,49.48,4,17.3,1,1354,8,249,3,0,96.63,745,4,13026,462,11,9,443,24.21,842,"""P. Gerkens""","""2021/2022""",6216,48,8,8701,148,34,0,"""2021-07-25""",1,18.27,6,218,225,59,17,2415,10,"""KV Mechelen v Royal Antwerp FC""","""Royal Antwerp FC""",1,1558,3,1874,139,369,true,100,11.08,2,1,22,33,17,89,35.57,47.15,661,47,34,0,25.7,1504,17,1,0,2744,56,0,16,"""Royal Antwerp FC""",11,17,2,26,292,260,358,2,2983,662,12,92,17,8,2,"""AM""",0,"""Pieter Gerkens""",121,52,5727,"""Midfield""","""1995-02-17""",6810,3,19,5,47,18,27,1,24,13.13,44999,1061,1,23,45,3432,"""BEL - Pro League - 2021/2022""",8,2021,2022,"""2021/2022""",16,"""BEL""","""Pro League""","""male""","""adult"""
136,9,31,201,119,3,2,50.07,1,15.07,0,489,5,249,3,1,96.19,476,5,9969,27,17,11,167,25.52,178,"""X. Mercier""","""2021/2022""",5127,41,9,10932,202,17,1,"""2021-07-24""",1,14.52,11,63,104,39,14,1101,14,"""Oud-Heverlee Leuven v SV Zulte…","""Oud-Heverlee Leuven""",1,1775,1,1660,81,267,true,77,12.32,1,2,25,20,10,77,29.59

In [27]:
# get the statsbomb levels
sb_cl = pl.read_csv('statsbomb-competition-levels.csv')

In [28]:
sb_cl.head()

competition_id,competition_name,competition_region_name,competition_level,competition_division_level
i64,str,str,i64,str
81,"""Liga Profesional""","""Argentina""",2,"""1"""
93,"""A-League""","""Australia""",3,"""1"""
47,"""Bundesliga""","""Austria""",2,"""1"""
1035,"""First Division B""","""Belgium""",3,"""2"""
46,"""Jupiler Pro League""","""Belgium""",2,"""1"""


In [29]:
# rename competition name for easy merge
sb_cl = sb_cl.rename({"competition_name":"sb_competition_name"})


In [30]:
sb_cl = sb_cl.drop(["competition_id", "competition_level"])

In [31]:
sb_cl.head()

sb_competition_name,competition_region_name,competition_division_level
str,str,str
"""Liga Profesional""","""Argentina""","""1"""
"""A-League""","""Australia""","""1"""
"""Bundesliga""","""Austria""","""1"""
"""First Division B""","""Belgium""","""2"""
"""Jupiler Pro League""","""Belgium""","""1"""


In [32]:
sb_sc = pl.read_csv('mapping-competition-seasons.csv')

In [33]:
sb_sc.head()

sb_competition_id,sb_competition_name,sb_season_id,sb_season_name,sc_competition_id,sc_competition_name,sc_season_id,sc_season_name,sc_competition_season_id,sc_competition_season_name
i64,str,i64,str,i64,str,i64,str,i64,str
2,"""Premier League""",317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
2,"""Premier League""",281,"""2023/2024""",1,"""Premier League""",28,"""2023/2024""",543,"""ENG - Premier League - 2023/20…"
2,"""Premier League""",235,"""2022/2023""",1,"""Premier League""",21,"""2022/2023""",387,"""ENG - Premier League - 2022/20…"
2,"""Premier League""",108,"""2021/2022""",1,"""Premier League""",8,"""2021/2022""",287,"""ENG - Premier League - 2021/20…"
2,"""Premier League""",90,"""2020/2021""",1,"""Premier League""",7,"""2020/2021""",147,"""ENG - Premier League - 2020/20…"


In [34]:
joined_sb_sc = sb_cl.join(sb_sc, on="sb_competition_name")

In [35]:
joined_sb_sc.head()

sb_competition_name,competition_region_name,competition_division_level,sb_competition_id,sb_season_id,sb_season_name,sc_competition_id,sc_competition_name,sc_season_id,sc_season_name,sc_competition_season_id,sc_competition_season_name
str,str,str,i64,i64,str,i64,str,i64,str,i64,str
"""Premier League""","""England""","""1""",2,317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
"""Premier League""","""Scotland""","""1""",2,317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
"""Premier League""","""Ukraine""","""1""",2,317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
"""Premier League""","""England""","""1""",2,281,"""2023/2024""",1,"""Premier League""",28,"""2023/2024""",543,"""ENG - Premier League - 2023/20…"
"""Premier League""","""Scotland""","""1""",2,281,"""2023/2024""",1,"""Premier League""",28,"""2023/2024""",543,"""ENG - Premier League - 2023/20…"


In [36]:
# extract columns containing skillcorner competition country and national division information, sc_competition_id
extracted_sb_sc = joined_sb_sc.select(["sc_competition_season_id", "competition_region_name", "competition_division_level"])

In [37]:
extracted_sb_sc = extracted_sb_sc.rename({"sc_competition_season_id":"competition_edition_id"})

In [38]:
extracted_sb_sc = extracted_sb_sc.with_columns(
    pl.concat_str(
        [
            pl.col("competition_region_name"),
            pl.col("competition_division_level").cast(str),
        ],
        separator=" ",
    ).alias("competition_region_division")
).drop(["competition_region_name", "competition_division_level"])

In [39]:
extracted_sb_sc.head()

competition_edition_id,competition_region_division
i64,str
895,"""England 1"""
895,"""Scotland 1"""
895,"""Ukraine 1"""
543,"""England 1"""
543,"""Scotland 1"""


In [40]:
final_df = work_df.join(extracted_sb_sc, on = "competition_edition_id")

In [41]:
final_df.shape

(508602, 128)

In [42]:
final_df.head()

Sprinting Distance TIP,Count HSR TIP 1,Count Medium Deceleration 1,HSR Distance 1,Sprinting Distance TIP 2,Count Sprint 1,Count High Deceleration TIP,Minutes 2,Count High Deceleration OTIP,Minutes OTIP 2,Count Sprint OTIP 2,Running Distance 1,Count High Deceleration 1,competition_edition_id,Count High Deceleration 2,Count High Acceleration TIP 2,Minutes,HSR Distance,Count High Acceleration,Distance,team_id,Sprinting Distance TIP 1,Count Medium Deceleration TIP 2,HSR Distance OTIP,Minutes TIP,Running Distance OTIP 1,short_name,season_name,Distance 2,Count Medium Acceleration 2,Count Medium Deceleration TIP 1,player_id,Sprinting Distance,Count HSR OTIP,Count High Acceleration OTIP 2,date,Count High Deceleration TIP 2,Minutes OTIP 1,Count HSR TIP 2,HSR Distance OTIP 2,HSR Distance OTIP 1,Count HSR,Count Medium Deceleration OTIP 2,Running Distance,Count Medium Acceleration TIP 2,match_name,team,Count Sprint TIP 1,Distance TIP 2,Count High Deceleration OTIP 1,Distance TIP 1,HSR Distance TIP 1,Running Distance TIP 1,quality_check,Count Medium Acceleration,Minutes TIP 2,Count High Deceleration TIP 1,Count High Acceleration 2,Count Medium Acceleration TIP,Count HSR 1,Count HSR OTIP 1,Count Medium Deceleration,Minutes OTIP,Minutes 1,Running Distance TIP,Count Medium Acceleration OTIP,Count Medium Deceleration OTIP,Count High Acceleration OTIP 1,PSV-99,Running Distance OTIP,Count Medium Deceleration TIP,Count Sprint OTIP 1,Count High Acceleration TIP 1,Distance OTIP 2,Sprinting Distance 2,Count High Acceleration OTIP,competition_id,team_name,Count High Deceleration,Count Medium Deceleration OTIP 1,Count Sprint OTIP,Count HSR 2,Running Distance TIP 2,HSR Distance TIP,HSR Distance 2,Count Sprint 2,Distance OTIP 1,Running Distance OTIP 2,Count Medium Acceleration TIP 1,Sprinting Distance 1,Count HSR OTIP 2,season_id,Count Sprint TIP,position,Count High Acceleration TIP,player_name,HSR Distance TIP 2,Count Medium Acceleration 1,Distance OTIP,group,player_birthdate,Distance 1,Count High Acceleration 1,Count HSR TIP,Count Sprint,Count Medium Deceleration 2,Sprinting Distance OTIP 2,Sprinting Distance OTIP 1,Count High Deceleration OTIP 2,Count Medium Acceleration OTIP 1,Minutes TIP 1,match_id,Running Distance 2,Count Sprint TIP 2,Count Medium Acceleration OTIP 2,Sprinting Distance OTIP,Distance TIP,name,season_id_right,season_start_year,season_end_year,season_name_right,competition_id_right,competition_area,competition_name,competition_gender,competition_age_group,competition_region_division
i64,i64,i64,i64,i64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,str,i64,f64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,bool,i64,f64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,str,i64,str,str,str,str,str
30,13,42,387,19,3,3,49.48,4,17.3,1,1354,8,249,3,0,96.63,745,4,13026,462,11,9,443,24.21,842,"""P. Gerkens""","""2021/2022""",6216,48,8,8701,148,34,0,"""2021-07-25""",1,18.27,6,218,225,59,17,2415,10,"""KV Mechelen v Royal Antwerp FC""","""Royal Antwerp FC""",1,1558,3,1874,139,369,true,100,11.08,2,1,22,33,17,89,35.57,47.15,661,47,34,0,25.7,1504,17,1,0,2744,56,0,16,"""Royal Antwerp FC""",11,17,2,26,292,260,358,2,2983,662,12,92,17,8,2,"""AM""",0,"""Pieter Gerkens""",121,52,5727,"""Midfield""","""1995-02-17""",6810,3,19,5,47,18,27,1,24,13.13,44999,1061,1,23,45,3432,"""BEL - Pro League - 2021/2022""",8,2021,2022,"""2021/2022""",16,"""BEL""","""Pro League""","""male""","""adult""","""Belgium 1"""
136,9,31,201,119,3,2,50.07,1,15.07,0,489,5,249,3,1,96.19,476,5,9969,27,17,11,167,25.52,178,"""X. Mercier""","""2021/2022""",5127,41,9,10932,202,17,1,"""2021-07-24""",1,14.52,11,63,104,39,14,1101,14,"""Oud-Heverlee Leuven v SV Zulte…","""Oud-Heverlee Leuven""",1,1775,1

In [43]:
final_df = final_df.select(sorted(final_df.columns))

In [44]:
# path to csv to write the dataframe to
csv_path = './data_summary.csv'

In [45]:
# writing  the dataframe to a csv
final_df.write_csv(csv_path)

In [ ]:
# extra columns taken to the next step
# competition_region_division
